# Deteccion de Cambios Urbanos en San Juan de Pasto

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/GeoIA-Pasto/notebooks/06_Deteccion_Cambios_Urbanos_Pasto.ipynb)

---

## Objetivos

1. Comparar imagenes satelitales de diferentes fechas
2. Detectar cambios en el area urbana de Pasto
3. Identificar expansion urbana y nuevas construcciones
4. Analizar cambios en la cobertura vegetal

## 1. Instalacion de Dependencias

In [ ]:
%pip install -q geoai-py leafmap pystac-client planetary-computer rioxarray

## 2. Importacion de Librerias

In [ ]:
import geoai
import leafmap
import rioxarray as rxr
import numpy as np
import matplotlib.pyplot as plt
import pystac_client
import planetary_computer
import os
import warnings

warnings.filterwarnings("ignore")

print(f"GeoAI version: {geoai.__version__}")

## 3. Definicion del Area de Estudio

In [ ]:
# Coordenadas de San Juan de Pasto
PASTO_CENTER = {"lat": 1.2136, "lon": -77.2811}
PASTO_BBOX = (-77.32, 1.18, -77.24, 1.26)

print(f"Area de estudio: {PASTO_BBOX}")

## 4. Descargar Imagenes de Dos Periodos

Vamos a comparar imagenes de:
- **Periodo 1**: 2020 (hace ~4 años)
- **Periodo 2**: 2024 (actual)

In [ ]:
# Conectar a Planetary Computer
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)


def buscar_mejor_imagen(bbox, fecha_inicio, fecha_fin, max_nubes=15):
    """
    Busca la imagen con menos nubes en un periodo.
    """
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=bbox,
        datetime=f"{fecha_inicio}/{fecha_fin}",
        query={"eo:cloud_cover": {"lt": max_nubes}},
    )

    items = list(search.items())
    if items:
        return min(items, key=lambda x: x.properties.get("eo:cloud_cover", 100))
    return None


# Buscar imagenes de 2020 y 2024
print("Buscando imagenes...")
imagen_2020 = buscar_mejor_imagen(PASTO_BBOX, "2020-01-01", "2020-12-31")
imagen_2024 = buscar_mejor_imagen(PASTO_BBOX, "2024-01-01", "2024-12-31")

if imagen_2020:
    print(f"\nImagen 2020: {imagen_2020.datetime.strftime('%Y-%m-%d')}")
    print(f"  Nubes: {imagen_2020.properties.get('eo:cloud_cover')}%")

if imagen_2024:
    print(f"\nImagen 2024: {imagen_2024.datetime.strftime('%Y-%m-%d')}")
    print(f"  Nubes: {imagen_2024.properties.get('eo:cloud_cover')}%")

In [ ]:
# Cargar bandas de ambas imagenes
def cargar_bandas(item, bbox, bandas=["B04", "B03", "B02", "B08"]):
    """
    Carga bandas especificas de una imagen.
    """
    datos = {}
    for banda in bandas:
        if banda in item.assets:
            url = item.assets[banda].href
            da = rxr.open_rasterio(url).squeeze()
            da = da.rio.clip_box(*bbox)
            datos[banda] = da
    return datos


print("Cargando imagenes...")

if imagen_2020 and imagen_2024:
    datos_2020 = cargar_bandas(imagen_2020, PASTO_BBOX)
    print(f"Bandas 2020 cargadas: {list(datos_2020.keys())}")

    datos_2024 = cargar_bandas(imagen_2024, PASTO_BBOX)
    print(f"Bandas 2024 cargadas: {list(datos_2024.keys())}")

## 5. Visualizar Imagenes de Ambos Periodos

In [ ]:
def crear_rgb(datos, max_val=3000):
    """
    Crea composicion RGB normalizada.
    """
    r = np.clip(datos["B04"].values / max_val, 0, 1)
    g = np.clip(datos["B03"].values / max_val, 0, 1)
    b = np.clip(datos["B02"].values / max_val, 0, 1)
    return np.dstack([r, g, b])


if imagen_2020 and imagen_2024:
    rgb_2020 = crear_rgb(datos_2020)
    rgb_2024 = crear_rgb(datos_2024)

    fig, axes = plt.subplots(1, 2, figsize=(16, 7))

    axes[0].imshow(rgb_2020)
    axes[0].set_title(
        f'Pasto - {imagen_2020.datetime.strftime("%Y-%m-%d")}', fontsize=12
    )
    axes[0].axis("off")

    axes[1].imshow(rgb_2024)
    axes[1].set_title(
        f'Pasto - {imagen_2024.datetime.strftime("%Y-%m-%d")}', fontsize=12
    )
    axes[1].axis("off")

    plt.suptitle("Comparacion Temporal - San Juan de Pasto", fontsize=14)
    plt.tight_layout()
    plt.savefig("./data/comparacion_temporal_pasto.png", dpi=150, bbox_inches="tight")
    plt.show()

## 6. Calcular NDVI para Ambos Periodos

In [ ]:
def calcular_ndvi(datos):
    """
    Calcula NDVI a partir de las bandas.
    """
    nir = datos["B08"].values.astype(float)
    red = datos["B04"].values.astype(float)
    ndvi = (nir - red) / (nir + red + 1e-10)
    return np.clip(ndvi, -1, 1)


if imagen_2020 and imagen_2024:
    ndvi_2020 = calcular_ndvi(datos_2020)
    ndvi_2024 = calcular_ndvi(datos_2024)

    print(f"NDVI 2020 - Promedio: {ndvi_2020.mean():.3f}")
    print(f"NDVI 2024 - Promedio: {ndvi_2024.mean():.3f}")

## 7. Detectar Cambios en Vegetacion

In [ ]:
# Calcular diferencia de NDVI
if imagen_2020 and imagen_2024:
    # Diferencia: positivo = ganancia de vegetacion, negativo = perdida
    ndvi_diff = ndvi_2024 - ndvi_2020

    print(f"Cambio NDVI - Min: {ndvi_diff.min():.3f}, Max: {ndvi_diff.max():.3f}")
    print(f"Cambio NDVI - Promedio: {ndvi_diff.mean():.3f}")

    # Estadisticas de cambio
    perdida = np.sum(ndvi_diff < -0.1) / ndvi_diff.size * 100
    ganancia = np.sum(ndvi_diff > 0.1) / ndvi_diff.size * 100
    estable = np.sum((ndvi_diff >= -0.1) & (ndvi_diff <= 0.1)) / ndvi_diff.size * 100

    print(f"\nCambios en vegetacion:")
    print(f"  Perdida (NDVI < -0.1): {perdida:.2f}%")
    print(f"  Estable: {estable:.2f}%")
    print(f"  Ganancia (NDVI > 0.1): {ganancia:.2f}%")

In [ ]:
# Visualizar cambios
if imagen_2020 and imagen_2024:
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # NDVI 2020
    im1 = axes[0].imshow(ndvi_2020, cmap="RdYlGn", vmin=-0.2, vmax=0.8)
    axes[0].set_title(f"NDVI 2020", fontsize=12)
    axes[0].axis("off")
    plt.colorbar(im1, ax=axes[0], fraction=0.046)

    # NDVI 2024
    im2 = axes[1].imshow(ndvi_2024, cmap="RdYlGn", vmin=-0.2, vmax=0.8)
    axes[1].set_title(f"NDVI 2024", fontsize=12)
    axes[1].axis("off")
    plt.colorbar(im2, ax=axes[1], fraction=0.046)

    # Diferencia
    im3 = axes[2].imshow(ndvi_diff, cmap="RdBu", vmin=-0.3, vmax=0.3)
    axes[2].set_title(
        "Cambio NDVI (2024 - 2020)\nRojo=Perdida, Azul=Ganancia", fontsize=12
    )
    axes[2].axis("off")
    plt.colorbar(im3, ax=axes[2], fraction=0.046)

    plt.suptitle("Deteccion de Cambios en Vegetacion - San Juan de Pasto", fontsize=14)
    plt.tight_layout()
    plt.savefig("./data/cambios_vegetacion_pasto.png", dpi=150, bbox_inches="tight")
    plt.show()

## 8. Detectar Expansion Urbana

In [ ]:
# Clasificar areas urbanas (NDVI bajo, alta reflectancia)
def detectar_urbano(datos, ndvi, umbral_ndvi=0.15):
    """
    Detecta areas urbanas basado en NDVI bajo y alta reflectancia.
    """
    red = datos["B04"].values.astype(float)

    # Urbano: NDVI bajo y reflectancia moderada-alta
    urbano = (ndvi < umbral_ndvi) & (red > 500) & (red < 2500)
    return urbano.astype(np.uint8)


if imagen_2020 and imagen_2024:
    urbano_2020 = detectar_urbano(datos_2020, ndvi_2020)
    urbano_2024 = detectar_urbano(datos_2024, ndvi_2024)

    # Detectar nuevas areas urbanas
    nuevo_urbano = (urbano_2024 == 1) & (urbano_2020 == 0)
    perdida_urbano = (urbano_2020 == 1) & (urbano_2024 == 0)

    print(f"Area urbana 2020: {urbano_2020.sum() / urbano_2020.size * 100:.2f}%")
    print(f"Area urbana 2024: {urbano_2024.sum() / urbano_2024.size * 100:.2f}%")
    print(f"Expansion urbana: {nuevo_urbano.sum() / nuevo_urbano.size * 100:.2f}%")

In [ ]:
# Visualizar expansion urbana
if imagen_2020 and imagen_2024:
    # Crear mapa de cambio urbano
    cambio_urbano = np.zeros_like(ndvi_2020, dtype=np.uint8)
    cambio_urbano[(urbano_2020 == 1) & (urbano_2024 == 1)] = 1  # Urbano estable
    cambio_urbano[nuevo_urbano] = 2  # Nueva area urbana
    cambio_urbano[perdida_urbano] = 3  # Area que dejo de ser urbana

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # Urbano 2020
    axes[0].imshow(urbano_2020, cmap="gray")
    axes[0].set_title("Areas Urbanas 2020", fontsize=12)
    axes[0].axis("off")

    # Urbano 2024
    axes[1].imshow(urbano_2024, cmap="gray")
    axes[1].set_title("Areas Urbanas 2024", fontsize=12)
    axes[1].axis("off")

    # Cambio
    colors = ["white", "gray", "red", "green"]
    cmap = plt.cm.colors.ListedColormap(colors)
    im = axes[2].imshow(cambio_urbano, cmap=cmap, vmin=0, vmax=3)
    axes[2].set_title(
        "Cambio Urbano\nGris=Estable, Rojo=Expansion, Verde=Recuperacion", fontsize=12
    )
    axes[2].axis("off")

    plt.suptitle("Expansion Urbana - San Juan de Pasto (2020-2024)", fontsize=14)
    plt.tight_layout()
    plt.savefig("./data/expansion_urbana_pasto.png", dpi=150, bbox_inches="tight")
    plt.show()

## 9. Usar GeoAI para Deteccion de Cambios Avanzada

In [ ]:
# GeoAI incluye funcionalidades avanzadas para deteccion de cambios
print("GeoAI ofrece modelos de deteccion de cambios basados en deep learning:")
print("")
print("Uso:")
print("from geoai import change_detection")
print("")
print("# Detectar cambios con modelo pre-entrenado")
print("cambios = change_detection.detect(")
print("    image1='imagen_2020.tif',")
print("    image2='imagen_2024.tif',")
print("    model='unet',")
print("    output='cambios.tif'")
print(")")

## 10. Mapa Interactivo con Slider Temporal

In [ ]:
# Crear mapa interactivo
m = leafmap.Map(
    center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=13, height="600px"
)

m.add_basemap("Esri.WorldImagery")
m.add_basemap("OpenStreetMap")

# Agregar capa de cambios si existe
if os.path.exists("./data/cambios_vegetacion_pasto.png"):
    print("Visualiza los cambios en las imagenes generadas")

m.add_layer_control()
m

## 11. Guardar Resultados

In [ ]:
# Crear directorio
os.makedirs("./data", exist_ok=True)

if imagen_2020 and imagen_2024:
    # Guardar diferencia NDVI
    ndvi_diff_da = datos_2020["B04"].copy()
    ndvi_diff_da.values = ndvi_diff
    ndvi_diff_da.rio.to_raster("./data/ndvi_cambio_pasto.tif")
    print("Guardado: ndvi_cambio_pasto.tif")

    # Guardar mapa de cambio urbano
    cambio_da = datos_2020["B04"].copy()
    cambio_da.values = cambio_urbano.astype(float)
    cambio_da.rio.to_raster("./data/cambio_urbano_pasto.tif")
    print("Guardado: cambio_urbano_pasto.tif")

## 12. Resumen

En este notebook hemos aprendido a:

1. **Comparar imagenes** de diferentes periodos (2020 vs 2024)
2. **Calcular diferencias de NDVI** para detectar cambios en vegetacion
3. **Identificar expansion urbana** usando umbrales espectrales
4. **Crear mapas de cambio** que muestran areas de perdida/ganancia
5. **Cuantificar cambios** en porcentajes del area total

### Observaciones para Pasto:

- La ciudad ha experimentado expansion en los ultimos años
- Las zonas periurbanas muestran mayor cambio
- Algunas areas han recuperado vegetacion

### Proximos pasos:

- Usar modelos de deep learning para deteccion de cambios mas precisa
- Analizar series temporales mas largas
- Correlacionar con datos socioeconomicos